# Understanding Targets

Targets are objects that a SoS step can input, output, or dependent on. They are usually files that are presented by filenames, but can also be other targets.

### `executable` target

`executable` targets are commands that should be accessible and executable by SoS. These targets are usually listed in the `depends` section of a SoS step. For example, SoS would stop if a command `fastqc` is not found.

In [2]:
[10]
input:     'a.fastq'
depends:   executable('fastqc')
sh:
    fastqc ${fastq_files}

No step to generate target a.fastq requested by default_10

`executable` target can also be output of a step but installing executables can be tricky because the commands should be installed to existing `$PATH` so that they can be immediately accessible by SoS. Because SoS automatically adds `~/.sos/bin` to `$PATH` (option `-b`), an environment-neutral way for on-the-fly installation is to install commands to this directory. For example

In [3]:
[10: provides=executable('fastqc')]
sh:
    # download fastqc
    # install fastqc
    # link fastqc to ~/.sos/bin

would be called if executable `fastqc` is not found.

You can also have finer control over which version of the command is eligible by checking the output of commands. The complete syntax is

```
executable(command, version='', test_command=None)
```
where `test_command` should be a command to execute, and `version` should be one or more strings that should be contained in the output of `test_command`. `test_command` can be ignored if it is identical to `command`.

For example, 

```
executable('STAR', ['2.4.0', '2.5.0'], test_command='STAR --version')
```

checks the output of command `STAR --version` and declare non-existence of command `STAR` if the output does not contain string `2.4.0` or `2.5.0`.

## `sos_variable` target

`sos_variable(name)` targets represent SoS variables that are created by a SoS step and shared to other steps. These targets can be used to provide information to other steps. For example,

In [4]:
[get_total_reads: shared='total_reads']
input:  'alignment/summary.txt'
with open(input[0]) as ifile:
    ...
    total_reads = int(...)


[100]
depends: sos_variable('total_reads')
output:  'report.txt'

with open(output[0]) as report:
    report.write('total reads: ${total_reads}')



No step to generate target alignment/summary.txt requested by get_total_reads

Step `100` needed some information extracted from output of another step. You can either parse the information in step `100` or use another step to provide the information. The latter is recommended because the information could be requested by multiple steps.

### `R_library` target

The `R_library` target represents a R library. If the libraries are not available, it will try to install it from [CRAN](https://cran.r-project.org/), [bioconductor](https://www.bioconductor.org/), or [github](https://github.com/). Github package name should be formatted as `repo/pkg`. A typical usage of this target would be

```
[10]
output: 'test.jpg'
depends: R_library('ggplot2')
R:
  library(ggplot2) 
  jpeg(${output!r})
  qplot(Sepal.Length, Petal.Length, data = iris, color = Species)
  dev.off()
```

`R_library` can also be used to check for specific versions of packages. For example:

```
R_library('edgeR', '3.12.0')
```
will result in a warning if edgeR version is not 3.12.0. You can specify multiple versions 

```
R_library('edgeR', ['3.12.0', '3.12.1'])
```

certain version or newer,
```
R_library('edgeR', '3.12.0+')
```

certain version or older
```
check_R_library('ggplot2', '1.0.0-')
```

The default R library repo is `http://cran.us.r-project.org`. It is possible to customize the repo for which a R library would be installed, for example:

```
R_library('Rmosek', repos = "http://download.mosek.com/R/7")
```

## `env_variable` target

SoS keeps tract of runtime environment and creates signatures of executed steps so that they do not have to be executed again. Some commands, especially shell scripts, could however behave differently with different environmental variables. To make sure a step would be re-executed with changing environments, you should list these variables as dependencies of the step. For example

```
[10]
depends:   env_variable('DEBUG')
sh:
    echo DEBUG is set to $DEBUG
```

## `dynamic` target

A `dynamic` target is a target that can only be determined when the step is actually executed. For example, the input of the following step would not be evaluated until step `20` is executed.

```
[10]
#
# a step that generate some text with dynamically determined names
#

[20]
input: dynamic('*.txt')


In this particular case, if you specify

```
[20]
input: '*.txt'

```

SoS would find all `.txt` files at the beginning of the execution of the workflow and feed them to step `20`.